# Fallstudie Asset Pricing 2021
## Tobias Schmid, Maximilian Kirsch und Niklas Vogel

### Import der benutzen Bibliotheken

In [229]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
import math

### Import der gegebenen Daten & Übersicht der Datensätze

In [10]:
monthly = pd.read_csv('/Users/NiklasVogel/Downloads/data/monthly_data_1964-2018.csv')
monthly.describe()

,company_id,year,month,day,price,ret,num_shares
count,3.136399e+06,3.136399e+06,3.136399e+06,3.136399e+06,3.098499e+06,3.092872e+06,3.132685e+06
mean,4.430639e+04,1.992430e+03,6.503874e+00,2.996407e+01,3.043882e+01,9.981725e-03,4.489867e+04
std,2.712886e+04,1.373249e+01,3.454350e+00,1.114489e+00,1.336032e+03,1.882013e-01,2.490105e+05
min,1.000000e+00,1.964000e+03,1.000000e+00,2.600000e+01,1.510000e-02,-1.000000e+00,2.000000e+00
25%,1.884800e+04,1.982000e+03,4.000000e+00,2.900000e+01,4.750000e+00,-6.716418e-02,2.920000e+03
50%,4.696100e+04,1.993000e+03,7.000000e+00,3.000000e+01,1.243000e+01,0.000000e+00,8.379000e+03
75%,6.986400e+04,2.003000e+03,1.000000e+01,3.100000e+01,2.489000e+01,7.142857e-02,2.671800e+04
max,8.343700e+04,2.018000e+03,1.200000e+01,3.100000e+01,3.260000e+05,4.700000e+01,2.920640e+07


In [11]:
monthly

,company_id,date,year,month,day,price,ret,num_shares
0,1,31jan1986,1986,1,31,4.375000,NaN,3680.0
1,1,28feb1986,1986,2,28,3.250000,-0.257143,3680.0
2,1,31mar1986,1986,3,31,4.437500,0.365385,3680.0
3,1,30apr1986,1986,4,30,4.000000,-0.098592,3793.0
4,1,30may1986,1986,5,30,3.109375,-0.222656,3793.0
...,...,...,...,...,...,...,...,...
3136394,83437,31aug2018,2018,8,31,301.660004,0.011806,170593.0
3136395,83437,28sep2018,2018,9,28,264.769989,-0.122290,171578.0
3136396,83437,31oct2018,2018,10,31,337.320007,0.274011,171733.0
3136397,83437,30nov2018,2018,11,30,350.480011,0.039013,171733.0


In [26]:
momentum = pd.read_csv('/Users/NiklasVogel/Downloads/data/F-F_Momentum_Factor.CSV')
momentum.describe()

,Unnamed: 0,Mom
count,1131.000000,1131.000000
mean,197369.087533,0.637728
std,2721.967916,4.710608
min,192701.000000,-52.270000
25%,195007.500000,-0.925000
50%,197402.000000,0.780000
75%,199708.500000,2.930000
max,202103.000000,18.360000


In [14]:
momentum

,Unnamed: 0,Mom
0,192701,0.44
1,192702,-1.32
2,192703,3.59
3,192704,4.17
4,192705,2.96
...,...,...
1126,202011,-12.25
1127,202012,-2.42
1128,202101,4.37
1129,202102,-7.68


In [63]:
research = pd.read_csv('/Users/NiklasVogel/Downloads/data/F-F_Research_Data_Factors.CSV')
research.describe()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
count,1137.000000,1137.000000,1137.000000,1137.000000,1137.000000
mean,197343.970976,0.684811,0.207740,0.336086,0.269894
std,2736.602825,5.348188,3.183763,3.509996,0.252052
min,192607.000000,-29.130000,-16.820000,-13.960000,-0.060000
25%,195003.000000,-1.970000,-1.590000,-1.390000,0.030000
50%,197311.000000,1.060000,0.080000,0.140000,0.220000
75%,199707.000000,3.650000,1.770000,1.720000,0.420000
max,202103.000000,38.850000,36.700000,35.460000,1.350000


In [64]:
research

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.30,-2.87,0.22
1,192608,2.64,-1.40,4.19,0.25
2,192609,0.36,-1.32,0.01,0.23
3,192610,-3.24,0.04,0.51,0.32
4,192611,2.53,-0.20,-0.35,0.31
...,...,...,...,...,...
1132,202011,12.47,5.48,2.11,0.01
1133,202012,4.63,4.81,-1.36,0.01
1134,202101,-0.03,7.19,2.85,0.00
1135,202102,2.78,2.11,7.08,0.00


### Aufbereitung & Vereinheitlichung der Datensätze

In [27]:
momentum.rename(columns={'Unnamed: 0': 'date'}, inplace = True)
momentum['date'] = pd.to_datetime(momentum['date'], format = '%Y%m')
momentum.set_index('date', inplace = True)
momentum

,Mom
date,
1927-01-01,0.44
1927-02-01,-1.32
1927-03-01,3.59
1927-04-01,4.17
1927-05-01,2.96
...,...
2020-11-01,-12.25
2020-12-01,-2.42
2021-01-01,4.37


In [65]:
research.rename(columns={'Unnamed: 0': 'date'}, inplace = True)
research['date'] = pd.to_datetime(research['date'], format = '%Y%m')
research.set_index('date', inplace = True)
research

,Mkt-RF,SMB,HML,RF
date,,,,
1926-07-01,2.96,-2.30,-2.87,0.22
1926-08-01,2.64,-1.40,4.19,0.25
1926-09-01,0.36,-1.32,0.01,0.23
1926-10-01,-3.24,0.04,0.51,0.32
1926-11-01,2.53,-0.20,-0.35,0.31
...,...,...,...,...
2020-11-01,12.47,5.48,2.11,0.01
2020-12-01,4.63,4.81,-1.36,0.01
2021-01-01,-0.03,7.19,2.85,0.00


Neues Dataframe 'factors', das alle Faktoren (Mkt-RF, SMB, HML, MOM) und RF beinhaltet

In [32]:
factors = pd.merge(research, momentum, left_index=True, right_index=True)

In [33]:
factors

,Mkt-RF,SMB,HML,RF,Mom
date,,,,,
1927-01-01,-0.06,-0.56,4.83,0.25,0.44
1927-02-01,4.18,-0.10,3.17,0.26,-1.32
1927-03-01,0.13,-1.60,-2.67,0.30,3.59
1927-04-01,0.46,0.43,0.60,0.25,4.17
1927-05-01,5.44,1.41,4.93,0.30,2.96
...,...,...,...,...,...
2020-11-01,12.47,5.48,2.11,0.01,-12.25
2020-12-01,4.63,4.81,-1.36,0.01,-2.42
2021-01-01,-0.03,7.19,2.85,0.00,4.37


### Import Datensatz Fama French Portfolios

In [44]:
skiplist = [*range(0,1098)] + [*range(2184,4661)]
fama_e = pd.read_csv('/Users/NiklasVogel/Downloads/10_Portfolios_Prior_60_13.CSV', skiprows = skiplist)
fama_e

,Unnamed: 0,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR
0,193101,29.68,23.22,19.48,17.59,11.08,10.47,11.86,8.62,9.17,9.42
1,193102,22.36,22.55,20.58,17.19,17.46,8.47,11.04,10.31,12.36,17.78
2,193103,-1.69,-3.43,-9.03,-3.19,-9.46,-9.72,-8.90,-7.61,-7.54,-7.04
3,193104,-21.26,-19.09,-15.53,-12.50,-14.63,-14.59,-15.37,-13.61,-14.14,-12.58
4,193105,-14.91,-9.17,-16.36,-15.42,-13.41,-13.17,-13.21,-14.79,-15.67,-17.05
...,...,...,...,...,...,...,...,...,...,...,...
1079,202012,12.19,10.15,8.63,8.08,6.89,7.43,5.67,5.48,5.89,8.37
1080,202101,36.58,19.56,7.53,4.27,2.76,0.51,0.60,0.20,0.79,2.74
1081,202102,13.14,11.95,9.86,9.70,10.60,9.29,8.03,7.39,7.24,5.05
1082,202103,0.19,2.89,4.31,4.92,5.56,5.51,5.37,4.29,2.54,-0.42


In [49]:
fama_e.rename(columns = {'Unnamed: 0': 'date'}, inplace = True)
fama_e['date'] = pd.to_datetime(fama_e['date'], format = '%Y%m')
fama_e = fama_e.set_index('date')

In [50]:
fama_e

,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR
date,,,,,,,,,,
1931-01-01,29.68,23.22,19.48,17.59,11.08,10.47,11.86,8.62,9.17,9.42
1931-02-01,22.36,22.55,20.58,17.19,17.46,8.47,11.04,10.31,12.36,17.78
1931-03-01,-1.69,-3.43,-9.03,-3.19,-9.46,-9.72,-8.90,-7.61,-7.54,-7.04
1931-04-01,-21.26,-19.09,-15.53,-12.50,-14.63,-14.59,-15.37,-13.61,-14.14,-12.58
1931-05-01,-14.91,-9.17,-16.36,-15.42,-13.41,-13.17,-13.21,-14.79,-15.67,-17.05
...,...,...,...,...,...,...,...,...,...,...
2020-12-01,12.19,10.15,8.63,8.08,6.89,7.43,5.67,5.48,5.89,8.37
2021-01-01,36.58,19.56,7.53,4.27,2.76,0.51,0.60,0.20,0.79,2.74
2021-02-01,13.14,11.95,9.86,9.70,10.60,9.29,8.03,7.39,7.24,5.05


### Regression zur Überprüfung der Strategie

Erstellen eines Dataframes mit allen Daten für den benötigten Zeitraum

In [89]:
test_frame = factors['1931-01-01':'1982-12-01'].join(fama_e['1931-01-01':'1982-12-01'])

In [90]:
test_frame['Lo PRIOR - RF'] = test_frame['Lo PRIOR'] - test_frame['RF']

In [91]:
test_frame

,Mkt-RF,SMB,HML,RF,Mom,Lo PRIOR,PRIOR 2,PRIOR 3,PRIOR 4,PRIOR 5,PRIOR 6,PRIOR 7,PRIOR 8,PRIOR 9,Hi PRIOR,Lo PRIOR - RF
date,,,,,,,,,,,,,,,,
1931-01-01,6.24,3.77,7.16,0.15,-10.82,29.68,23.22,19.48,17.59,11.08,10.47,11.86,8.62,9.17,9.42,29.53
1931-02-01,10.88,3.38,1.61,0.04,-14.37,22.36,22.55,20.58,17.19,17.46,8.47,11.04,10.31,12.36,17.78,22.32
1931-03-01,-6.43,3.07,-3.66,0.13,9.46,-1.69,-3.43,-9.03,-3.19,-9.46,-9.72,-8.90,-7.61,-7.54,-7.04,-1.82
1931-04-01,-9.98,-4.62,-3.94,0.08,11.59,-21.26,-19.09,-15.53,-12.50,-14.63,-14.59,-15.37,-13.61,-14.14,-12.58,-21.34
1931-05-01,-13.24,5.17,-6.58,0.09,9.10,-14.91,-9.17,-16.36,-15.42,-13.41,-13.17,-13.21,-14.79,-15.67,-17.05,-15.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982-08-01,11.14,-4.09,1.00,0.76,-3.52,5.63,8.14,7.45,7.80,6.50,7.92,6.85,8.77,7.75,8.61,4.87
1982-09-01,1.29,2.89,0.33,0.51,4.17,4.13,3.39,3.49,3.22,4.29,4.23,4.04,4.51,2.99,1.97,3.62
1982-10-01,11.30,2.35,-3.73,0.59,0.03,14.25,11.35,11.37,10.39,12.15,12.19,12.04,12.33,14.14,15.70,13.66


Test mit gleichgewichtetem Portfolio

In [92]:
X = test_frame[['Mkt-RF']] 
Y = test_frame['Lo PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X) 

date
1931-01-01    11.468758
1931-02-01    19.172483
1931-03-01    -9.567061
1931-04-01   -15.461075
1931-05-01   -20.873606
                ...    
1982-08-01    19.604157
1982-09-01     3.250344
1982-10-01    19.869803
1982-11-01     8.862110
1982-12-01     2.021733
Length: 624, dtype: float64

In [93]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Lo PRIOR - RF   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.560
Method:                 Least Squares   F-statistic:                     795.1
Date:                Mon, 14 Jun 2021   Prob (F-statistic):          2.57e-113
Time:                        13:44:38   Log-Likelihood:                -2227.7
No. Observations:                 624   AIC:                             4459.
Df Residuals:                     622   BIC:                             4468.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1086      0.347      3.196      0.001       0.427       1.790
Mkt-RF         1.6603      0.059     28.198      0.000       1.545       1.776
==============================================================================
Omnibus:                      560.054   Durbin-Watson:                   2.004
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19424.039
Skew:                           3.891   Prob(JB):                         0.00
Kurtosis:                      29.201   Cond. No.                         5.93
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Regression mit Carhart-Factor Model

In [94]:
X = test_frame[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = test_frame['Lo PRIOR - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)

date
1931-01-01    22.499525
1931-02-01    20.889071
1931-03-01    -5.272469
1931-04-01   -23.208612
1931-05-01   -10.477188
                ...    
1982-08-01     4.717499
1982-09-01     6.466712
1982-10-01    11.042824
1982-11-01    10.293167
1982-12-01     0.614300
Length: 624, dtype: float64

In [95]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Lo PRIOR - RF   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     1282.
Date:                Mon, 14 Jun 2021   Prob (F-statistic):          7.75e-298
Time:                        13:44:44   Log-Likelihood:                -1789.3
No. Observations:                 624   AIC:                             3589.
Df Residuals:                     619   BIC:                             3611.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5045      0.179      2.822      0.005       0.153       0.856
Mkt-RF         0.8946      0.035     25.616      0.000       0.826       0.963
SMB            1.8276      0.057     32.129      0.000       1.716       1.939
HML            1.0346      0.054     19.009      0.000       0.928       1.142
Mom           -0.1955      0.042     -4.637      0.000      -0.278      -0.113
==============================================================================
Omnibus:                      348.123   Durbin-Watson:                   2.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6690.453
Skew:                           2.049   Prob(JB):                         0.00
Kurtosis:                      18.509   Cond. No.                         7.45
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Selbst implementierte Long-Term Reversal Strategie

Ansatz: Sortieren in "Winner" und "Loser", indem die Renditen für den Zeitraum von vor fünf Jahren bis vor einem Jahr addiert werden und ein Top 10% und Flop 10% Portfolio gebaut wird. Portfolio wird gleichgewichtet.

In [96]:
monthly

,company_id,date,year,month,day,price,ret,num_shares
0,1,31jan1986,1986,1,31,4.375000,NaN,3680.0
1,1,28feb1986,1986,2,28,3.250000,-0.257143,3680.0
2,1,31mar1986,1986,3,31,4.437500,0.365385,3680.0
3,1,30apr1986,1986,4,30,4.000000,-0.098592,3793.0
4,1,30may1986,1986,5,30,3.109375,-0.222656,3793.0
...,...,...,...,...,...,...,...,...
3136394,83437,31aug2018,2018,8,31,301.660004,0.011806,170593.0
3136395,83437,28sep2018,2018,9,28,264.769989,-0.122290,171578.0
3136396,83437,31oct2018,2018,10,31,337.320007,0.274011,171733.0
3136397,83437,30nov2018,2018,11,30,350.480011,0.039013,171733.0


In [174]:
monthly_new = monthly.loc[monthly['year'] >= 2010]

In [175]:
monthly_new

,company_id,date,year,month,day,price,ret,num_shares
306,2,29jan2010,2010,1,29,10.060000,-0.018932,4361.0
307,2,26feb2010,2010,2,26,10.008400,-0.000656,4361.0
308,2,31mar2010,2010,3,31,10.170000,0.020643,4361.0
309,2,30apr2010,2010,4,30,11.390000,0.124385,6070.0
310,2,28may2010,2010,5,28,11.400000,0.004829,6071.0
...,...,...,...,...,...,...,...,...
3136394,83437,31aug2018,2018,8,31,301.660004,0.011806,170593.0
3136395,83437,28sep2018,2018,9,28,264.769989,-0.122290,171578.0
3136396,83437,31oct2018,2018,10,31,337.320007,0.274011,171733.0
3136397,83437,30nov2018,2018,11,30,350.480011,0.039013,171733.0


In [176]:
monthly_new['date'] = pd.to_datetime(monthly_new['date'], format = '%d%b%Y')

<ipython-input-176-882b132856ab>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_new['date'] = pd.to_datetime(monthly_new['date'], format = '%d%b%Y')


In [178]:
monthly_new.dtypes

company_id             int64
date          datetime64[ns]
year                   int64
month                  int64
day                    int64
price                float64
ret                  float64
num_shares           float64
dtype: object

In [179]:
monthly_new

,company_id,date,year,month,day,price,ret,num_shares
306,2,2010-01-29,2010,1,29,10.060000,-0.018932,4361.0
307,2,2010-02-26,2010,2,26,10.008400,-0.000656,4361.0
308,2,2010-03-31,2010,3,31,10.170000,0.020643,4361.0
309,2,2010-04-30,2010,4,30,11.390000,0.124385,6070.0
310,2,2010-05-28,2010,5,28,11.400000,0.004829,6071.0
...,...,...,...,...,...,...,...,...
3136394,83437,2018-08-31,2018,8,31,301.660004,0.011806,170593.0
3136395,83437,2018-09-28,2018,9,28,264.769989,-0.122290,171578.0
3136396,83437,2018-10-31,2018,10,31,337.320007,0.274011,171733.0
3136397,83437,2018-11-30,2018,11,30,350.480011,0.039013,171733.0


In [180]:
len(monthly_new['company_id'].unique())

5974

In [192]:
master_dict = {2015: {},
               2016: {},
               2017: {},
               2018: {},
                    }

for year in range(2015, 2019):
    for month in range(1, 13):
        print(f'{year}, {month}')
        past_av_ret = []
        for company in monthly_new.loc[(monthly_new['date'] < datetime.datetime(year-1, month, 1))]['company_id'].unique():
            help_df = monthly_new.loc[(monthly_new['company_id'] == company) & (monthly_new['date'] >  datetime.datetime(year-5, month, 1)) & (monthly_new['date'] < datetime.datetime(year-1, month, 1))]
            av_ret = help_df['ret'].mean()
            past_av_ret.append([av_ret, company])
        past_av_ret.sort(key=lambda x: x[0])
        master_dict[year][month] = {}
        master_dict[year][month]['past returns'] = past_av_ret
        master_dict[year][month]['past tops'] = past_av_ret[int(1/10*len(past_av_ret)):]
        master_dict[year][month]['past flops'] = past_av_ret[0:int(1/10*len(past_av_ret))]

2015, 1
2015, 2
2015, 3
2015, 4
2015, 5
2015, 6
2015, 7
2015, 8
2015, 9
2015, 10
2015, 11
2016, 1
2016, 2
2016, 3
2016, 4
2016, 5
2016, 6
2016, 7
2016, 8
2016, 9
2016, 10
2016, 11
2017, 1
2017, 2
2017, 3
2017, 4
2017, 5
2017, 6
2017, 7
2017, 8
2017, 9
2017, 10
2017, 11


Renditen der Tops und Flops

In [201]:
for year in master_dict.keys():
    for month in master_dict[year].keys():
        print(f'{year}, {month}')
        top_returns = []
        for element in master_dict[year][month]['past tops']:
            df = monthly_new.loc[(monthly_new['company_id'] == element[1]) & (monthly_new['year']==year) & (monthly_new['month']==month)]
            if df.empty:
                pass
            else:
                top_returns.append(df['ret'].values[0])
        master_dict[year][month]['tops todays returns'] = top_returns

2015, 1
2015, 2
2015, 3
2015, 4
2015, 5
2015, 6
2015, 7
2015, 8
2015, 9
2015, 10
2015, 11
2015, 12
2016, 1
2016, 2
2016, 3
2016, 4
2016, 5
2016, 6
2016, 7
2016, 8
2016, 9
2016, 10
2016, 11
2016, 12
2017, 1
2017, 2
2017, 3
2017, 4
2017, 5
2017, 6
2017, 7
2017, 8
2017, 9
2017, 10
2017, 11
2017, 12
2018, 1
2018, 2
2018, 3
2018, 4
2018, 5
2018, 6
2018, 7
2018, 8
2018, 9
2018, 10
2018, 11
2018, 12


In [249]:
mean_tops = []
for year in master_dict.keys():
    for month in master_dict[year].keys():
        sum_ = 0
        counter = 0
        for element in master_dict[year][month]['tops todays returns']:
            if math.isnan(element):
                pass
            else:
                sum_ += element
                counter += 1
        mean_tops.append([datetime.datetime(year, month, 1), sum_/counter])

In [241]:
for year in master_dict.keys():
    for month in master_dict[year].keys():
        flop_returns = []
        for element in master_dict[year][month]['past flops']:
            df = monthly_new.loc[(monthly_new['company_id'] == element[1]) & (monthly_new['year']==year) & (monthly_new['month']==month)]
            if df.empty:
                pass
            else:
                flop_returns.append(df['ret'].values[0])
        master_dict[year][month]['flops todays returns'] = flop_returns

In [245]:
mean_flops = []
for year in master_dict.keys():
    for month in master_dict[year].keys():
        sum_ = 0
        counter = 0
        for element in master_dict[year][month]['flops todays returns']:
            if math.isnan(element):
                pass
            else:
                sum_ += element
                counter += 1
        mean_flops.append([datetime.datetime(year, month, 1), sum_/counter])

In [251]:
mean_flops

[[datetime.datetime(2015, 1, 1, 0, 0), -0.03135428604473682],
 [datetime.datetime(2015, 2, 1, 0, 0), 0.06646123084303802],
 [datetime.datetime(2015, 3, 1, 0, 0), -0.009768594130653264],
 [datetime.datetime(2015, 4, 1, 0, 0), -0.01363588655808081],
 [datetime.datetime(2015, 5, 1, 0, 0), 0.015891736303571435],
 [datetime.datetime(2015, 6, 1, 0, 0), 0.008118659250559284],
 [datetime.datetime(2015, 7, 1, 0, 0), -0.02681536204338393],
 [datetime.datetime(2015, 8, 1, 0, 0), -0.03281726393319203],
 [datetime.datetime(2015, 9, 1, 0, 0), -0.06400174318453868],
 [datetime.datetime(2015, 10, 1, 0, 0), 0.045404885861842104],
 [datetime.datetime(2015, 11, 1, 0, 0), 0.011194203036402573],
 [datetime.datetime(2015, 12, 1, 0, 0), -0.05058029974669605],
 [datetime.datetime(2016, 1, 1, 0, 0), -0.1053801507889182],
 [datetime.datetime(2016, 2, 1, 0, 0), 0.0024055757176165786],
 [datetime.datetime(2016, 3, 1, 0, 0), 0.06170156698743721],
 [datetime.datetime(2016, 4, 1, 0, 0), 0.045897318405333294],
 [date

In [254]:
mean_flopsNtops = []
for i in range(0,len(mean_flops)):
    mean_flopsNtops.append([mean_flops[i][0], mean_flops[i][1], mean_tops[i][1]])

In [256]:
mean_flopsNtops

[[datetime.datetime(2015, 1, 1, 0, 0),
  -0.03135428604473682,
  -0.034519057294668844],
 [datetime.datetime(2015, 2, 1, 0, 0),
  0.06646123084303802,
  0.0661525033676369],
 [datetime.datetime(2015, 3, 1, 0, 0),
  -0.009768594130653264,
  0.0018973058760165262],
 [datetime.datetime(2015, 4, 1, 0, 0),
  -0.01363588655808081,
  0.002876775616855489],
 [datetime.datetime(2015, 5, 1, 0, 0),
  0.015891736303571435,
  0.007543166835897973],
 [datetime.datetime(2015, 6, 1, 0, 0),
  0.008118659250559284,
  -0.00392461833923637],
 [datetime.datetime(2015, 7, 1, 0, 0),
  -0.02681536204338393,
  -0.032181502477718005],
 [datetime.datetime(2015, 8, 1, 0, 0),
  -0.03281726393319203,
  -0.04482704762352939],
 [datetime.datetime(2015, 9, 1, 0, 0),
  -0.06400174318453868,
  -0.05561860501650153],
 [datetime.datetime(2015, 10, 1, 0, 0),
  0.045404885861842104,
  0.05394573288506217],
 [datetime.datetime(2015, 11, 1, 0, 0),
  0.011194203036402573,
  0.026052442225477063],
 [datetime.datetime(2015, 12, 

In [270]:
returns_reversal_frame = pd.DataFrame(mean_flopsNtops, columns = ['date', 'return flops', 'return tops'])
returns_reversal_frame = returns_reversal_frame.set_index('date')
returns_reversal_frame

,return flops,return tops
date,,
2015-01-01,-0.031354,-0.034519
2015-02-01,0.066461,0.066153
2015-03-01,-0.009769,0.001897
2015-04-01,-0.013636,0.002877
2015-05-01,0.015892,0.007543
2015-06-01,0.008119,-0.003925
2015-07-01,-0.026815,-0.032182
2015-08-01,-0.032817,-0.044827
2015-09-01,-0.064002,-0.055619


In [272]:
test_frame_new = factors['2015-01-01':'2018-12-01'].join(returns_reversal_frame)
test_frame_new['return flops'] = test_frame_new['return flops']*100
test_frame_new['return tops'] = test_frame_new['return tops']*100
test_frame_new['return flops - RF'] = test_frame_new['return flops'] - test_frame_new['RF']
test_frame_new['return tops - RF'] = test_frame_new['return tops'] - test_frame_new['RF']
test_frame_new

,Mkt-RF,SMB,HML,RF,Mom,return flops,return tops,return flops - RF,return tops - RF
date,,,,,,,,,
2015-01-01,-3.11,-0.51,-3.56,0.00,3.81,-3.135429,-3.451906,-3.135429,-3.451906
2015-02-01,6.14,0.52,-1.81,0.00,-2.83,6.646123,6.615250,6.646123,6.615250
2015-03-01,-1.12,3.02,-0.41,0.00,3.00,-0.976859,0.189731,-0.976859,0.189731
2015-04-01,0.59,-3.04,1.88,0.00,-7.32,-1.363589,0.287678,-1.363589,0.287678
2015-05-01,1.36,0.89,-1.10,0.00,6.03,1.589174,0.754317,1.589174,0.754317
2015-06-01,-1.53,2.85,-0.74,0.00,3.07,0.811866,-0.392462,0.811866,-0.392462
2015-07-01,1.54,-4.09,-4.21,0.00,10.29,-2.681536,-3.218150,-2.681536,-3.218150
2015-08-01,-6.04,0.44,2.73,0.00,-2.09,-3.281726,-4.482705,-3.281726,-4.482705
2015-09-01,-3.07,-2.61,0.39,0.00,5.35,-6.400174,-5.561861,-6.400174,-5.561861


In [284]:
test_frame_new = test_frame_new.drop(test_frame_new.index[len(test_frame_new)-1])

In [285]:
test_frame_new

,Mkt-RF,SMB,HML,RF,Mom,return flops,return tops,return flops - RF,return tops - RF
date,,,,,,,,,
2015-01-01,-3.11,-0.51,-3.56,0.00,3.81,-3.135429,-3.451906,-3.135429,-3.451906
2015-02-01,6.14,0.52,-1.81,0.00,-2.83,6.646123,6.615250,6.646123,6.615250
2015-03-01,-1.12,3.02,-0.41,0.00,3.00,-0.976859,0.189731,-0.976859,0.189731
2015-04-01,0.59,-3.04,1.88,0.00,-7.32,-1.363589,0.287678,-1.363589,0.287678
2015-05-01,1.36,0.89,-1.10,0.00,6.03,1.589174,0.754317,1.589174,0.754317
2015-06-01,-1.53,2.85,-0.74,0.00,3.07,0.811866,-0.392462,0.811866,-0.392462
2015-07-01,1.54,-4.09,-4.21,0.00,10.29,-2.681536,-3.218150,-2.681536,-3.218150
2015-08-01,-6.04,0.44,2.73,0.00,-2.09,-3.281726,-4.482705,-3.281726,-4.482705
2015-09-01,-3.07,-2.61,0.39,0.00,5.35,-6.400174,-5.561861,-6.400174,-5.561861


Test Flops

In [286]:
X = test_frame_new[['Mkt-RF']] 
Y = test_frame_new['return flops - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X) 

date
2015-01-01   -3.444400
2015-02-01    5.803589
2015-03-01   -1.454833
2015-04-01    0.254796
2015-05-01    1.024628
2015-06-01   -1.864743
2015-07-01    1.204589
2015-08-01   -6.373763
2015-09-01   -3.404409
2015-10-01    7.413239
2015-11-01    0.224802
2015-12-01   -2.504604
2016-01-01   -6.103821
2016-02-01   -0.415059
2016-03-01    6.623410
2016-04-01    0.584724
2016-05-01    1.444537
2016-06-01   -0.385065
2016-07-01    3.614065
2016-08-01    0.164815
2016-09-01   -0.085131
2016-10-01   -2.354637
2016-11-01    4.523867
2016-12-01    1.484528
2017-01-01    1.604502
2017-02-01    3.234148
2017-03-01   -0.165113
2017-04-01    0.754687
2017-05-01    0.724693
2017-06-01    0.444754
2017-07-01    1.534517
2017-08-01   -0.175111
2017-09-01    2.174378
2017-10-01    1.914435
2017-11-01    2.784245
2017-12-01    0.724693
2018-01-01    5.243710
2018-02-01   -3.984282
2018-03-01   -2.684565
2018-04-01   -0.045139
2018-05-01    2.314348
2018-06-01    0.144819
2018-07-01    2.854230
2018-0

In [287]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      return flops - RF   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     80.05
Date:                Tue, 15 Jun 2021   Prob (F-statistic):           1.50e-11
Time:                        10:56:32   Log-Likelihood:                -106.54
No. Observations:                  47   AIC:                             217.1
Df Residuals:                      45   BIC:                             220.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3351      0.359     -0.933      0.356      -1.058       0.388
Mkt-RF         0.9998      0.112      8.947      0.000       0.775       1.225
==============================================================================
Omnibus:                        0.866   Durbin-Watson:                   2.399
Prob(Omnibus):                  0.649   Jarque-Bera (JB):                0.944
Skew:                           0.248   Prob(JB):                        0.624
Kurtosis:                       2.513   Cond. No.                         3.34
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [291]:
X = test_frame_new[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = test_frame_new['return flops - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X)

date
2015-01-01   -4.057570
2015-02-01    5.787083
2015-03-01    0.678583
2015-04-01   -0.581373
2015-05-01    0.652262
2015-06-01    0.163164
2015-07-01   -3.868762
2015-08-01   -4.370123
2015-09-01   -5.508592
2015-10-01    5.604204
2015-11-01    2.721678
2015-12-01   -4.863006
2016-01-01   -7.595560
2016-02-01    1.025256
2016-03-01    7.321751
2016-04-01    2.447604
2016-05-01    0.714142
2016-06-01   -0.455947
2016-07-01    5.572482
2016-08-01    1.932759
2016-09-01    1.601360
2016-10-01   -4.929783
2016-11-01    9.456372
2016-12-01    1.865354
2017-01-01    0.650107
2017-02-01    1.513935
2017-03-01    0.763134
2017-04-01    1.096020
2017-05-01   -1.721056
2017-06-01    2.326689
2017-07-01    0.071708
2017-08-01   -1.950483
2017-09-01    5.818368
2017-10-01   -0.343026
2017-11-01    2.201620
2017-12-01    0.084094
2018-01-01    1.570636
2018-02-01   -3.719044
2018-03-01    1.107838
2018-04-01    0.970690
2018-05-01    5.222911
2018-06-01    1.359788
2018-07-01    1.246666
2018-0

In [292]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      return flops - RF   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.906
Method:                 Least Squares   F-statistic:                     111.6
Date:                Tue, 15 Jun 2021   Prob (F-statistic):           8.53e-22
Time:                        11:22:29   Log-Likelihood:                -72.909
No. Observations:                  47   AIC:                             155.8
Df Residuals:                      42   BIC:                             165.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1232      0.184     -0.670      0.507      -0.494       0.248
Mkt-RF         0.8509      0.061     13.937      0.000       0.728       0.974
SMB            0.7501      0.073     10.316      0.000       0.603       0.897
HML            0.0846      0.086      0.985      0.330      -0.089       0.258
Mom           -0.1586      0.062     -2.572      0.014      -0.283      -0.034
==============================================================================
Omnibus:                        0.815   Durbin-Watson:                   1.962
Prob(Omnibus):                  0.665   Jarque-Bera (JB):                0.888
Skew:                          -0.275   Prob(JB):                        0.642
Kurtosis:                       2.612   Cond. No.                         4.12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Test Tops

In [288]:
X = test_frame_new[['Mkt-RF']] 
Y = test_frame_new['return tops - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X) 

date
2015-01-01   -3.749773
2015-02-01    6.364243
2015-03-01   -1.573893
2015-04-01    0.295834
2015-05-01    1.137757
2015-06-01   -2.022190
2015-07-01    1.334570
2015-08-01   -6.953456
2015-09-01   -3.706037
2015-10-01    8.124629
2015-11-01    0.263031
2015-12-01   -2.721970
2016-01-01   -6.658236
2016-02-01   -0.436749
2016-03-01    7.260837
2016-04-01    0.656658
2016-05-01    1.596988
2016-06-01   -0.403947
2016-07-01    3.969682
2016-08-01    0.197427
2016-09-01   -0.075925
2016-10-01   -2.557959
2016-11-01    4.964682
2016-12-01    1.640724
2017-01-01    1.771933
2017-02-01    3.554187
2017-03-01   -0.163397
2017-04-01    0.842537
2017-05-01    0.809735
2017-06-01    0.503581
2017-07-01    1.695395
2017-08-01   -0.174331
2017-09-01    2.395175
2017-10-01    2.110890
2017-11-01    3.062154
2017-12-01    0.809735
2018-01-01    5.751935
2018-02-01   -4.340213
2018-03-01   -2.918783
2018-04-01   -0.032189
2018-05-01    2.548252
2018-06-01    0.175559
2018-07-01    3.138692
2018-0

In [289]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       return tops - RF   R-squared:                       0.727
Model:                            OLS   Adj. R-squared:                  0.721
Method:                 Least Squares   F-statistic:                     119.6
Date:                Tue, 15 Jun 2021   Prob (F-statistic):           2.93e-14
Time:                        10:56:48   Log-Likelihood:                -101.32
No. Observations:                  47   AIC:                             206.6
Df Residuals:                      45   BIC:                             210.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3493      0.321     -1.087      0.283      -0.997       0.298
Mkt-RF         1.0934      0.100     10.935      0.000       0.892       1.295
==============================================================================
Omnibus:                        0.917   Durbin-Watson:                   2.499
Prob(Omnibus):                  0.632   Jarque-Bera (JB):                0.890
Skew:                           0.137   Prob(JB):                        0.641
Kurtosis:                       2.385   Cond. No.                         3.34
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [293]:
X = test_frame_new[['Mkt-RF', 'SMB', 'HML', 'Mom   ']] 
Y = test_frame_new['return tops - RF']
X = sm.add_constant(X) 
model = sm.OLS(Y, X).fit()
model.predict(X) 

date
2015-01-01    -4.330485
2015-02-01     6.339316
2015-03-01     0.516715
2015-04-01    -0.560816
2015-05-01     0.806085
2015-06-01    -0.034186
2015-07-01    -3.552299
2015-08-01    -5.024277
2015-09-01    -5.735074
2015-10-01     6.347317
2015-11-01     2.705661
2015-12-01    -5.005174
2016-01-01    -8.115970
2016-02-01     0.943153
2016-03-01     7.918721
2016-04-01     2.434443
2016-05-01     0.897166
2016-06-01    -0.453196
2016-07-01     5.864688
2016-08-01     1.898282
2016-09-01     1.563150
2016-10-01    -5.072292
2016-11-01     9.744744
2016-12-01     2.005883
2017-01-01     0.838911
2017-02-01     1.872538
2017-03-01     0.736701
2017-04-01     1.177787
2017-05-01    -1.563914
2017-06-01     2.332354
2017-07-01     0.278414
2017-08-01    -1.886305
2017-09-01     5.936767
2017-10-01    -0.066628
2017-11-01     2.491632
2017-12-01     0.175346
2018-01-01     2.198362
2018-02-01    -4.067163
2018-03-01     0.764321
2018-04-01     0.956072
2018-05-01     5.405106
2018-06-01 

In [294]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       return tops - RF   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                     354.6
Date:                Tue, 15 Jun 2021   Prob (F-statistic):           9.21e-32
Time:                        11:23:21   Log-Likelihood:                -48.395
No. Observations:                  47   AIC:                             106.8
Df Residuals:                      42   BIC:                             116.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1456      0.109     -1.334      0.190      -0.366       0.075
Mkt-RF         0.9495      0.036     26.199      0.000       0.876       1.023
SMB            0.7308      0.043     16.933      0.000       0.644       0.818
HML            0.0816      0.051      1.599      0.117      -0.021       0.185
Mom           -0.1493      0.037     -4.080      0.000      -0.223      -0.075
==============================================================================
Omnibus:                        0.476   Durbin-Watson:                   1.573
Prob(Omnibus):                  0.788   Jarque-Bera (JB):                0.488
Skew:                          -0.222   Prob(JB):                        0.783
Kurtosis:                       2.770   Cond. No.                         4.12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""